In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import cv2
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
import tensorflow.keras.backend as K
from tensorflow.keras.applications import resnet
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from skimage import io, color
from tensorflow.keras.layers import Lambda
import torch.nn.functional as F
import torch
from tensorflow.keras.models import load_model
import pandas as pd
import argparse
from PIL import Image
from numpy import genfromtxt
import time
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
import shutil
import random
import csv

## Crop Out Components from Image

In [ ]:
with open('Dataset/result_arv_model_11_11_2022.json', 'r') as file:
    data = json.load(file)

j = 0 #index for Body
for i in range(len(data['ClassList'][j]['Data'])):

    fileName = data['ClassList'][j]['Data'][i]['ViewInfo']['ViewName']
    fileType = fileName+'_WhiteLight.jpg'
    newX = data['ClassList'][j]['Data'][i]['Xmin'] - data['ClassList'][j]['Data'][i]['ViewInfo']['ViewX']
    newY = data['ClassList'][j]['Data'][i]['Ymin'] - data['ClassList'][j]['Data'][i]['ViewInfo']['ViewY']
    width = data['ClassList'][j]['Data'][i]['Width']
    height = data['ClassList'][j]['Data'][i]['Height']
    x1 = newX + (width)
    y1 = newY + (height)
    
    image = cv2.imread('Dataset/Sample/Sample_3/W_N/'+fileName+'_W_N.jpg')
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    crop_image = img_rgb[newY:int(y1), newX:int(x1)]
    
    cv2.imwrite('Dataset/Cropped_Body/img_'+str(i)+'.jpg', crop_image)
print("DONE")

## Cropped Image Data to Numpy

### Without Resize

In [ ]:
temp_list = [] #__________ for anchor
temp_nameList = []
test_list = [] #__________ for test set
test_nameList = []
img_label = [] #__________ img label
file_nameList = []

folder_directory1 = 'Imageset/Classified/a_T1/' #___________ Folder path
ori_OR_sharp1 = input("1:Ori 2:Sharp")
rgb_OR_bgr1 = input("1: RGB, 2:BGR")
kernel1 = np.array([[0, -1, 0],
                           [-1, 5,-1],
                           [0, -1, 0]])
count = 0
for classified_folder in os.listdir(folder_directory1): #-------- looping 
    img_label2 = []
    test_nameList2 = []
    temp_nameList2 = []
    file_nameList.append(classified_folder)
    #____________ RESET LIST ___________#
    temp_list =[]
    test_list =[]
    for img in os.listdir(folder_directory1+classified_folder): #-------- looping images in 12362362364
        
        #img_label2 = []
        img_split_name = img.split('_')  # ------------------------------------ split file name
        img_proc1 = cv2.imread(folder_directory1+classified_folder+'/'+img) #---- read img
        if img == '.ipynb_checkpoints':
            print('ignored .ipynb_checkpoints')
        else:
            if ori_OR_sharp1 == "2":   # ---------------- 1: Ori 2: Sharpen
                img_proc1 = cv2.filter2D(src=img_proc1, ddepth=-1, kernel=kernel1)   
            else:
                img_proc1 = img_proc1

            if rgb_OR_bgr1 == "1":  # ---------------- 1: RGB 2: BGR
                img_proc1 = cv2.cvtColor(img_proc1, cv2.COLOR_BGR2RGB)
            
            #_______________ ANCHOR ______________#
            if img_split_name[0] == 'anchor':   
                temp_list.append(img_proc1)
                temp_nameList2.append(img)
                #print("anchor")
            #_______________ DEFECT ______________#
            elif img_split_name[0] == 'defect':
                test_list.append(img_proc1)
                test_nameList2.append(img)
                img_label2.append(0)
             #_______________ GOOD ______________#
            elif img_split_name[0] == 'good':
                test_list.append(img_proc1)
                test_nameList2.append(img)
                img_label2.append(1)
                
    temp_to_nparray = np.asarray(temp_list) #___________ list to numpy
    test_to_nparray = np.asarray(test_list)

    temp_output = temp_to_nparray/255
    test_output = test_to_nparray/255


    with open('Imageset/numpy/temp/set_'+str(count)+'.npy','wb') as temp_file: #_________save file
        np.save(temp_file, temp_output)
    with open('Imageset/numpy/test/set_'+str(count)+'.npy','wb') as test_file:
        np.save(test_file, test_output)
    print(test_output.shape)
            
            
             
    temp_nameList.append(temp_nameList2)
    test_nameList.append(test_nameList2)
    img_label.append(img_label2)
    count += 1
print('DONE')

### Resize

In [ ]:
###################################    IMAGE RESIZE  ################################# 

temp_list = [] #__________ for anchor
temp_nameList = []
test_list = [] #__________ for test set
test_nameList = []
img_label = [] #__________ img label
file_nameList = []

folder_directory1 = 'Imageset/Classified/a_T1/' #___________ Folder path
ori_OR_sharp1 = input("1:Ori 2:Sharp")
rgb_OR_bgr1 = input("1: RGB, 2:BGR")
kernel1 = np.array([[0, -1, 0],
                    [-1, 5,-1],
                    [0, -1, 0]])

npyTempFolder = 'Imageset/numpy/ori_bgr_temp7'
npyTestFolder = 'Imageset/numpy/ori_bgr_test7'
checknpyFolder  = os.path.isdir(npyTempFolder)
if checknpyFolder == False:
        os.mkdir(str(npyTempFolder))
        os.mkdir(str(npyTestFolder))
        
count = 0
for classified_folder in os.listdir(folder_directory1): #-------- looping 231274112387 in folder
    img_label2 = []
    test_nameList2 = []
    temp_nameList2 = []
    file_nameList.append(classified_folder)
    #____________ RESET LIST ___________#
    temp_list =[]
    test_list =[]
    temp_list2 =[]
    test_list2 =[]
    for img in os.listdir(folder_directory1+classified_folder): #-------- looping images in 12362362364
        
        #img_label2 = []
        img_split_name = img.split('_')  # ------------------------------------ split file name
        img_proc1 = cv2.imread(folder_directory1+classified_folder+'/'+img) #---- read img
        if img == '.ipynb_checkpoints':
            print('ignored .ipynb_checkpoints')
        else:
            if ori_OR_sharp1 == "2":   # ---------------- 1: Ori 2: Sharpen
                img_proc1 = cv2.filter2D(src=img_proc1, ddepth=-1, kernel=kernel1)   
            else:
                img_proc1 = img_proc1

            if rgb_OR_bgr1 == "1":  # ---------------- 1: RGB 2: BGR
                img_proc1 = cv2.cvtColor(img_proc1, cv2.COLOR_BGR2RGB)
                
            ori_image_height, ori_image_width, ori_channels = img_proc1.shape
            
            if ori_image_height < 75 or ori_image_width < 75:  #resize image with any side <75 only
                
                if ori_image_height < ori_image_width:
                    aspect_ratio = ori_image_width/ori_image_height
                    new_image_width = 75*aspect_ratio
                    img_proc11 = cv2.resize(img_proc1, (int(new_image_width),75))
                elif ori_image_height > ori_image_width:
                    aspect_ratio = ori_image_height/ori_image_width
                    new_image_height = 75*aspect_ratio
                    img_proc11 = cv2.resize(img_proc1, (75,int(new_image_height)))
            else:
                img_proc11 = img_proc1
            
            #_______________ ANCHOR ______________#
            if img_split_name[0] == 'anchor':   
                temp_list.append(img_proc11)
                temp_nameList2.append(img)
                temp_to_norma = img_proc11
                temp_output = np.array(temp_to_norma)
                temp_list2.append(temp_output)
                #print("anchor")
            #_______________ DEFECT ______________#
            elif img_split_name[0] == 'defect':
                test_list.append(img_proc11)
                test_nameList2.append(img)
                img_label2.append(0)
                test_to_norma = img_proc11
                test_output = np.array(test_to_norma)
                test_list2.append(test_output)
             #_______________ GOOD ______________#
            elif img_split_name[0] == 'good':
                test_list.append(img_proc11)
                test_nameList2.append(img)
                img_label2.append(1)
                test_to_norma = img_proc11
                test_output = np.array(test_to_norma)
                test_list2.append(test_output)
    
    with open(npyTempFolder+'/set_'+str(count)+'.npy','wb') as temp_file: #_________save file
        np.save(temp_file, temp_list2)
    with open(npyTestFolder+'/set_'+str(count)+'.npy','wb') as test_file:
        np.save(test_file, test_list2)
        
    print("TEST:  ", temp_output.shape)
    print("Template: ",test_output.shape)
            
            
             
    temp_nameList.append(temp_nameList2)
    test_nameList.append(test_nameList2)
    img_label.append(img_label2)
    count += 1


csv_fileName = input('filename: ')
with open(csv_fileName, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    for row in range(len(img_label)):
        label_a = (img_label[row])
        writer.writerow(label_a)
        
print('DONE')

## Distance Type

In [ ]:
def manhattan_distance(anchor, compare):
    return K.sum(K.abs(anchor - compare), axis=1, keepdims=True)
    #return sum(abs(a - b) for a, b in zip(point1, point2))

def minkowski_distance(point1, point2, p):
    return K.sum(K.abs(point1 - point2)**p, axis=-1)**(1/p)

def cosine_similarity(x, y, p):
    print(p)
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return K.sum(x * y, axis=-1)

def jaccard_similarity(x, y, p):
    print(p)
    intersection = K.sum(K.abs(x * y), axis=-1)
    union = K.sum(K.abs(x) + K.abs(y), axis=-1) - intersection
    return intersection / union


## Pre-Trained Network

In [ ]:
icp_V3_noTOP = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation='softmax'
)

dnn_169_noTOP = tf.keras.applications.densenet.DenseNet169(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation='softmax'
)

IR_V2_noTOP = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation='softmax',)

    #_________________get the model with prediction layer______________#
ef_V2 = tf.keras.applications.efficientnet_v2.EfficientNetV2S(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax',
    include_preprocessing=True
)

#_________________get the model without prediction layer______________#
ef_V2_noTOP = tf.keras.applications.efficientnet_v2.EfficientNetV2S(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax',
    include_preprocessing=True
)

#_________________get the layer before prediction that missed out______________#
ef_v2_avgpool = ef_V2.get_layer('avg_pool')
ef_v2_dropout = ef_V2.get_layer('top_dropout')


#_________________combined them as a new model______________#
ef_v2_final = ef_v2_avgpool(ef_V2_noTOP.output)
ef_v2_final = ef_v2_dropout(ef_v2_final)

new_ef_V2_model = Model(inputs=ef_V2_noTOP.inputs, outputs=ef_v2_final, name = "EF_V2")

#_________________importing without the prediction will miss out some of the fully connected layers______________#
#_________________importing with the prediction will need to specify the input size______________#

## Get Distance Score

In [ ]:
distance_Type = input("Distance Type to use: \n1: Minkowski Distance\n2: Cosine Similarity\n3: Jaccard Similarity\n")
model_Choice =  input("Model Type to use: \n1: DenseNet\n2: EfficientNet\n3: Inception\n4: InceptionResNet\n")                   
test_Type = input("\n\n1: 1 to 1 Image\n2: 1 to All Image\n3: ALL to ALL")
test_set = input("\ntest set: \n")
black = [1,2,5,7]                      
d = []
d2 = []
if int(model_Choice) == 1:
    Model_toPrint = ("DenseNet")
    model_type = dnn_169_noTOP
elif int(model_Choice) == 2:
    Model_toPrint = ("EfficientNet")
    model_type = new_ef_V2_model
elif int(model_Choice) == 3:
    Model_toPrint = ("Inception V3")
    model_type = icp_V3_noTOP
elif int(model_Choice) == 4:
    Model_toPrint = ("InceptionResNet V2")
    model_type = IR_V2_noTOP
    
if int(distance_Type) == 1:
    Distance_toPrint = ("Minkowski Distance")
    p_value = input("P value:\n1: Mahathan\n2: Euclidean\n")
    distance_type = minkowski_distance
elif int(distance_Type) == 2:
    Distance_toPrint = ("Cosine Similarity")
    distance_type = cosine_similarity
elif int(distance_Type) == 3:
    Distance_toPrint = ("Jaccard Similarity")
    distance_type = jaccard_similarity
#elif int(distance_Type) == 3:
    #distance_type = DistanceLayer()
print(Model_toPrint, "\t",Distance_toPrint, "\n")
imgnp_test = np.load('Dataset/'+test_set+'.npy', allow_pickle=True)
imgnp = np.load('Dataset/temp111.npy', allow_pickle=True)                      
if int(test_Type) == 1:
                      
    #_________________Load Images______________#
    
    img_no = input("\ntemplate img no: ")
    img_no2 = input("\nquery img no: ")                  
    #_________________tiling images, model only takes vector size of 4, image only have 3______________#
    imgt1 = (np.tile(imgnp[int(img_no)],(1,1,1,1)))
    imgt2 = (np.tile(imgnp[xx],(1,1,1,1)))

    #_________________getting the vectors______________#
    pred1 = model_type.predict(imgt1)
    pred2 = model_type.predict(imgt2)

    #_________________getting the distance______________#
    distances = distance_type(pred1,pred2,int(p_value))
    distances2 = DistanceLayer()(pred1,pred2)                  
    print(distances)
    plt.imshow(imgnp[int(img_no)])
    plt.show()
    plt.imshow(imgnp[int(img_no2)])
    plt.show()
                      
elif int(test_Type) == 2:
                      
    img_no3 = input("\nquery img no: ")
        
    for xx in range(len(imgnp_test)):
                      
        #_________________tiling images, model only takes vector size of 4, image only have 3______________#
        imgt1 = (np.tile(imgnp[int(img_no3)],(1,1,1,1)))
        imgt2 = (np.tile(imgnp_test[xx],(1,1,1,1)))

        #_________________getting the vectors______________#
        pred1 = model_type.predict(imgt1)
        pred2 = model_type.predict(imgt2)

        #_________________getting the distance______________#
        distances = distance_type(pred1,pred2,int(p_value))
        distances2 = DistanceLayer()(pred1,pred2) 
        print(distances)
        distance_numpy = distances.numpy()
        d.append(distance_numpy[0])
        print(xx)
        plt.imshow(imgnp_test[xx])
        plt.show()

elif int(test_Type) == 3:
    
    for yy in range(len(imgnp)):
        d3 = []
        for xx in range(len(imgnp_test)):
            #_________________tiling images, model only takes vector size of 4, image only have 3______________#
            imgt1 = (np.tile(imgnp[yy],(1,1,1,1)))
            imgt2 = (np.tile(imgnp_test[xx],(1,1,1,1)))

            #_________________getting the vectors______________#
            pred1 = model_type.predict(imgt1)
            pred2 = model_type.predict(imgt2)

            #_________________getting the distance______________#
            distances = distance_type(pred1,pred2,int(p_value))
            print(distances)
            distance_numpy = distances.numpy()
            d.append(distance_numpy[0])
            d3.append(distance_numpy[0])
            #d3.append(distance_numpy[0])
            print(xx)
            plt.imshow(imgnp_test[xx])
            plt.show()
        d2.append(d3)
else:
    test_set2 = input("test set 2: ")
    imgnp_test2 = np.load('Dataset/'+test_set2+'.npy', allow_pickle=True)
    d4 = []
    d5 = []
    
    for yy in range(len(imgnp)):
        
        d3 = []
        for xx in range(len(imgnp_test)):
            #_________________tiling images, model only takes vector size of 4, image only have 3______________#
            imgt1 = (np.tile(imgnp[yy],(1,1,1,1)))
            imgt2 = (np.tile(imgnp_test[xx],(1,1,1,1)))

            #_________________getting the vectors______________#
            pred1 = model_type.predict(imgt1)
            pred2 = model_type.predict(imgt2)

            #_________________getting the distance______________#
            distances = distance_type(pred1,pred2,int(p_value))
            print(distances)
            distance_numpy = distances.numpy()
            d.append(distance_numpy[0])
            d3.append(distance_numpy[0])

        d2.append(d3)
        
    for yyy in range(len(imgnp)):
        
        d6 = []
        for xxx in range(len(imgnp_test2)):
            #_________________tiling images, model only takes vector size of 4, image only have 3______________#
            imgt1s = (np.tile(imgnp[yyy],(1,1,1,1)))
            imgt2s = (np.tile(imgnp_test2[xxx],(1,1,1,1)))

            #_________________getting the vectors______________#
            pred1s = model_type.predict(imgt1s)
            pred2s = model_type.predict(imgt2s)

            #_________________getting the distance______________#
            distancess = distance_type(pred1s,pred2s,int(p_value))
            print(distancess)
            distance_numpys = distancess.numpy()
            d4.append(distance_numpys[0])
            d6.append(distance_numpys[0])
            
        d5.append(d6)

## Get AUC

In [ ]:
auc_max_list = []
auc_not_list = []
#______________________________   Temp vs Test   ______________________________#

model_Choice =  input("Model Type to use: \n1: DenseNet\n2: EfficientNet\n3: Inception\n4: InceptionResNet\n")                   
p_value = 2

if int(model_Choice) == 1:
    Model_toPrint = ("DenseNet")
    model_type = dnn_169_noTOP
elif int(model_Choice) == 2:
    Model_toPrint = ("EfficientNet")
    model_type = new_ef_V2_model
elif int(model_Choice) == 3:
    Model_toPrint = ("Inception V3")
    model_type = icp_V3_noTOP
elif int(model_Choice) == 4:
    Model_toPrint = ("InceptionResNet V2")
    model_type = IR_V2_noTOP

d_list = []
d_list2=[]

start_time = time.time()
for temp_test in tqdm (range(251), desc="Loading..."):
    if temp_test == 177 or temp_test == 190:
        print("ignored")
    else:
        temp2 = np.load('Imageset/numpy/ori_bgr_temp7/set_'+str(temp_test)+'.npy')
        test2 = np.load('Imageset/numpy/ori_bgr_test7/set_'+str(temp_test)+'.npy')
        #true_label = img_label[temp_test]
        #print(true_label)
        for yy in range(len(temp2)):
            d_list=[]
            for xx in range(len(test2)):
                #plt.imshow(temp2[yy])
                #plt.show()
                #plt.imshow(test2[xx])
                #plt.show()
                img1 = (np.tile(temp2[yy],(1,1,1,1)))
                img2 = (np.tile(test2[xx],(1,1,1,1)))

                pred4 = model_type.predict(img1, verbose = 0)
                pred5 = model_type.predict(img2, verbose = 0)
                #pred5 = icp_V3_noTOP.predict(img2)

                            #_________________getting the distance______________#
                distances = minkowski_distance(pred4,pred5,p_value)
                #print(distances)
                distance_numpys = distances.numpy()
                #d4.append(distance_numpys[0])
                d_list.append(distance_numpys[0])
            d_list2.append(d_list)
    pass

print("END TIME FOR Temp Vs Test: --- %s seconds ---" % (time.time() - start_time))        
    #print(d_list)
    #print(temp2.shape)
    #print(temp_test)
    
#______________________________   Temp vs Temp   ______________________________#

start_time2 = time.time()    
d_list_MAX = []
d_list2_MAX=[]
#for i in tqdm (range (1), desc="Loading..."):
    #for temp_test_MAX in range(251):
for temp_test_MAX in tqdm (range(251), desc="Loading..."):
    if temp_test_MAX == 177 or temp_test_MAX == 190:
        print("ignored")
    else:
        temp2_MAX = np.load('Imageset/numpy/ori_bgr_temp7/set_'+str(temp_test_MAX)+'.npy')
        test2_MAX = np.load('Imageset/numpy/ori_bgr_temp7/set_'+str(temp_test_MAX)+'.npy')
        #true_label = img_label_removed[temp_test]
        #print(true_label)
        for yy_MAX in range(len(temp2_MAX)):
            d_list_MAX=[]
            for xx_MAX in range(len(test2_MAX)):
                #plt.imshow(temp2[yy])
                #plt.show()
                #plt.imshow(test2[xx])
                #plt.show()
                img1_MAX = (np.tile(temp2_MAX[yy_MAX],(1,1,1,1)))
                img2_MAX = (np.tile(test2_MAX[xx_MAX],(1,1,1,1)))

                pred4_MAX = model_type.predict(img1_MAX, verbose = 0)
                pred5_MAX = model_type.predict(img2_MAX, verbose = 0)
                #pred5 = icp_V3_noTOP.predict(img2)

                            #_________________getting the distance______________#
                distances_MAX = minkowski_distance(pred4_MAX,pred5_MAX,p_value)
                #print(distances)
                distance_numpys_MAX = distances_MAX.numpy()
                #d4.append(distance_numpys[0])
                d_list_MAX.append(distance_numpys_MAX[0])
            d_list2_MAX.append(d_list_MAX)
    #print(d_list_MAX)
    #print(temp2_MAX.shape)
    #print(temp_test_MAX)
    pass
print("END TIME FOR Temp Vs Temp: --- %s seconds ---" % (time.time() - start_time2))


#______________________________   AUC without Threshold   ______________________________#

average_list2 = []
fprs_list = []
tprs_list = []
jjj = 0
auc = 0
kt = 0
ign = 0
score75 = []
score75no = []
for jjj in range(0,747,3):
    average_list = []
    auc2 = 0
    
    for jj in range(len(d_list2[0])):
        average_score = (d_list2[jjj][jj]+d_list2[jjj+1][jj]+d_list2[jjj+2][jj])/3
        average_list.append(average_score)

    
    average_list2.append(average_list)
    
for kkk in range(len(average_list2)):
    if img_label_removed[kkk] == [1,1,1,1] or img_label_removed[kkk] == [0,0,0,0]:
        ign = ign + 1
        auc_not_list.append(2)
    else:
        kt = kt + 1
        auc2 = roc_auc_score(img_label_removed[kkk], average_list2[kkk])
        reverse_auc = 1-auc2
        fpr, tpr, _ = roc_curve(img_label_removed[kkk], average_list2[kkk])
        fprs_list.append(fpr)
        tprs_list.append(tpr)
        #auc_max_list.append(auc_max) 
        auc_not_list.append(reverse_auc)
        auc = auc + reverse_auc
        if reverse_auc < 0.76:
            score75.append(reverse_auc)
            score75no.append(kkk)
        
print(len(img_label_removed))
print('_______________________________ WIHOUT THRESHOLD _______________________________')
print('AVERAGE AUC: ', auc/kt)
print('AVERAGE AUC WITH ALL TRUE: ', (auc+ign)/249)
print('ALL TRUE: ', ign)

#______________________________   AUC with MAX Threshold   ______________________________#

########################################################################   FIND MAX SCORE FOR AS THRESHOLD  #########################################
d_max3 = []
d_max2 = []
d_max1 = []
for dist in (d_list2_MAX):
    d_max1.append(max(dist))
    
for lll in range(0,747,3):
    d_max2 = []
    d_max = 0 
    pp = 0
    for nn in range(3): 
        if d_max < d_max1[nn+lll]:

            d_max = d_max1[nn+lll]

        else:
            pp = pp + 1
    d_max2.append(d_max)

    d_max3.append(d_max2)


########################################################################   MAX THRESHOLD   ##################################################################################
pred_label = []

count_pred = 0
for comp in average_list2:
    pred_label2 = []
    for score in comp:
        if score < d_max3[count_pred]:
            pred_label2.append(1)
        elif score > d_max3[count_pred]:
            pred_label2.append(0)
    count_pred = count_pred + 1
    pred_label.append(pred_label2)
    
########################################################################   FIND AUC   ##################################################################################
ign_max = 0
auc_count_max = 0
auc_max = 0
low_auc_imgno = []
low_auc_score = []
fprs_max_list = []
tprs_max_list = []
for qqq in range(249):
    if img_label_removed[qqq] == [1,1,1,1] or img_label_removed[qqq] == [0,0,0,0]:
        #print("ignore")
        ign_max = ign_max + 1
    else:
        auc_count_max = auc_count_max + 1
        auc2_max = roc_auc_score(img_label_removed[qqq], pred_label[qqq])
        
        fpr_max, tpr_max, _ = roc_curve(img_label_removed[qqq], pred_label[qqq])
        fprs_max_list.append(fpr_max)
        tprs_max_list.append(tpr_max)
        auc_max_list.append(auc2_max)
        auc_max = auc_max + auc2_max
        if auc2_max < 0.4:
            low_auc_score.append(auc2_max)
            low_auc_imgno.append(qqq)
print('_______________________________ MAX THRESHOLD _______________________________')
print(len(img_label_removed))
print('AVERAGE AUC: ', auc_max/auc_count_max)
print('AVERAGE AUC WITH ALL TRUE: ', (auc_max+ign_max)/249)

########################################################################   export to CSV   ##################################################################################
